# Fund21. TF2 API
가장 간단한 이미지 분류 문제를 풀어가면서 TF2(TensorFlow V2)를 활용한 모델링을 진행해보자.  
TensorFlow2에서 딥러닝 모델을 작성하는 방법에는 크게 `Sequential`, `Functional`, `Module Subclassing`의 3가지가 있다. 

### 1) Sequential Model
```python
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)
```  

- 입력부터 출력까지 레이어를 sequential하게 add해서 쌓아나간다. 이 방식은 초보자가 접근하기에 매우 쉽다는 것이 장점이다. 
- 모델의 입력과 출력이 여러 개인 경우에는 적합하지 않은 모델링 방식이다. Sequential 모델은 반드시 입력 1가지, 출력 1가지를 전제로 한다.

### 2) Functional API
```python
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.fit(x,y, epochs=10, batch_size=32)
```  
- `Sequntial Model`을 활용하는 것과 다른 점은 `keras.Model`을 사용한다는 점이다. 이는 `Sequential Model`을 쓰는 것 보다 더 일반적인 접근이다. 
- `Functional API`를 활용하면 더 자유로운 모델링을 진행할 수 있다. 함수형으로 모델을 구성하는 것, 즉 **입력과 출력을 규정해 모델 전체를 규정**한다. 
- input이 될 수 있는 텐서가 여러 개가 될 수 있다. 레이어들을 자유롭게 엮어 출력까지 규정하면 model이 inputs와 outputs로 규정된다.

### 3) Subclassing
```python
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
    
    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        
        return x
    
model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)
```
- `Subclassing`을 활용하면 가장 자유로운 모델링을 할 수 있다. `keras.Model`을 상속 받은 모델 클래스를 만들기 때문에 `Functional`한 접근과 차이가 없다. 
- `__init__()` 메서드 안에서 레이어 구성을 정의하고, `call()` 메서드 안에서 레이어 간 forward propagation을 구현하다. 

## MNIST (1) - Sequential API 활용

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

11501568/11490434 [==============================] - 0s 0us/step
60000 10000


In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [4]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 32s 3ms/step - loss: 0.1067 - accuracy: 0.9677
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0348 - accuracy: 0.9888
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0189 - accuracy: 0.9938
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0122 - accuracy: 0.9962
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0086 - accuracy: 0.9970
313/313 - 1s - loss: 0.0509 - accuracy: 0.9860


[0.05088239535689354, 0.9860000014305115]

## MNIST (2) - Functional API 활용

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [7]:
inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [8]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1066 - accuracy: 0.9675
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0359 - accuracy: 0.9884
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0195 - accuracy: 0.9937
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0134 - accuracy: 0.9955
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0095 - accuracy: 0.9970
313/313 - 1s - loss: 0.0512 - accuracy: 0.9866


[0.051226891577243805, 0.9865999817848206]

## MNIST (3) - Subclassing 활용

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [11]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()

In [12]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1052 - accuracy: 0.9677
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0346 - accuracy: 0.9894
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0188 - accuracy: 0.9938
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0108 - accuracy: 0.9965
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0096 - accuracy: 0.9968
313/313 - 1s - loss: 0.0527 - accuracy: 0.9863


[0.052677400410175323, 0.986299991607666]

## CIFAR-100 - Sequential API 활용

In [13]:
import tensorflow as tf
from tensorflow import keras

In [14]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

169017344/169001437 [==============================] - 3s 0us/step
50000 10000


In [15]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6051 - accuracy: 0.1592
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9178 - accuracy: 0.2803
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6087 - accuracy: 0.3440
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3882 - accuracy: 0.3894
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2146 - accuracy: 0.4278
313/313 - 1s - loss: 2.5726 - accuracy: 0.3637


[2.572640895843506, 0.3637000024318695]

## CIFAR-100 - Functional API 활용

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [19]:
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2, 2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6176 - accuracy: 0.1569
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9037 - accuracy: 0.2841
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6094 - accuracy: 0.3432
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4059 - accuracy: 0.3841
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2502 - accuracy: 0.4168
313/313 - 1s - loss: 2.5962 - accuracy: 0.3591


[2.596174716949463, 0.35910001397132874]

## CIFAR-100 - Subclassing 활용

In [21]:
import tensorflow as tf
from tensorflow import keras

In [22]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [23]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2, 2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2, 2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()

In [24]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.6635 - accuracy: 0.1481
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9801 - accuracy: 0.2667
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6933 - accuracy: 0.3219
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4845 - accuracy: 0.3694
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3188 - accuracy: 0.4020
313/313 - 1s - loss: 2.6142 - accuracy: 0.3481


[2.6141819953918457, 0.3481000065803528]

## GrdientTape 활용
Tensorflow에서 제공하는 `tf.GradientTape`는 forward pass로 진행된 모든 연산의 중간 레이어 값을 tape에 기록하고, 이를 이용해 gradient를 계산한 후, tape를 폐기하는 기능을 수행한다. 이전 스텝에서 진행했던 학습을 `tf.GradientTape`를 이용한 것으로 변형해보자. 

In [25]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


- 아래 코드는 `tape.gradient()`를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후, `optimizer.apply_gradients()`를 통해 발생한 그래디언트가 업데이트해야 할 파라미터 `model.trainable_variables`를 지정해주는 과정을 기술한 것이다.

In [26]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    # backpropagation
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [27]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 4.5027
Epoch 1: last batch loss = 4.4263
Epoch 2: last batch loss = 4.4792
Epoch 3: last batch loss = 4.5164
Epoch 4: last batch loss = 4.5672
It took 71.02432513237 seconds


- `tf.GradientTape()`을 활용하면 `model.compile()`과 `model.fit()` 안에 감추어져 있던 한 스텝의 학습 단계를 끄집어내서 자유롭게 재구성할 수 있게 된다. 
- 지도학습 방식과 다른 강화학습 또는 GAN(Generative Advasarial Network)의 학습을 위해서는 `train_step` 메서드의 재구성이 필수적이므로 `tf.GradientTape()`의 활용법을 꼭 숙지하자. 

In [28]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 1s 910ms/step


0.0401